# Machine Learning Matching(机器学习匹配)

In [1]:
# Import modules 导入模块
import pandas as pd
import numpy as np
import csv
import py_entitymatching as em
import os

## 读取样本文件

In [2]:
filepath1 = "datasets/DBLP-Scholar/DBLP1.csv"
filepath2 = "datasets/DBLP-Scholar/Scholar.csv"
filepath3 = "datasets/DBLP-Scholar/DBLP-Scholar_perfectMapping.csv"

dblp = pd.read_csv(filepath1, encoding="iso8859-1")
scholar = pd.read_csv(filepath2)

In [3]:
S = em.read_csv_metadata("datasets/DBLP-Scholar/candidates_sample.csv", key='_id',
                         ltable=dblp, rtable=scholar,
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
S.head(10)

,Unnamed: 0,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year,label
0,9,9,conf/sigmod/2000,SHYtwo2v-LAJ,"proceedings of the 2000 acm sigmod international conference on management of data, may 16-18, 20...",NaN,2000,w3qs: a query system for the world-wide webâ» in proceedings of the 21st international conferenc...,"d konopnicki, o shmueli",0,0
1,53,53,conf/sigmod/2002,url:http://portal.acm.org/citation.cfm%3Fid%3D582192,"proceedings of the 2002 acm sigmod international conference on management of data, madison, wisc...",NaN,2002,source international conference on management of data archive,m stonebraker,1992,0
2,61,61,conf/sigmod/2003,CpwYam_LYyEJ,"proceedings of the 2003 acm sigmod international conference on management of data, san diego, ca...",NaN,2003,on predicting data cache behavior for real {time systems. in proceedings of the acm sigplan work...,"c ferdinand, r wilhelm",0,0
3,77,77,conf/sigmod/2003,url:http://portal.acm.org/citation.cfm%3Fid%3D276304%26dl%3DACM%26coll%3Dportal%26CFID%3D1111111...,"proceedings of the 2003 acm sigmod international conference on management of data, san diego, ca...",NaN,2003,source international conference on management of data archive,"a tiwary, m franklin",1998,0
4,102,102,conf/sigmod/AbiteboulCM95,XTa-ElbpjZ8J,a database interface for file updates,"s abiteboul, s cluet, t milo",1995,a logical view of structured les,"s abiteboul, s cluet, t milo",0,0
5,115,115,conf/sigmod/AcharyaAFZ95,h2z51QPXcSkJ,broadcast disks: data management for asymmetric communications environments,"s acharya, r alonso, m franklin, s zdonik",1995,broadca. rt l) ikr: data management for uymmetrir communications environment,"s acharya, r alonso, mj franklin, s zdonik",1995,1
6,155,155,conf/sigmod/AdaliSS99,Cbwq3JSxrSAJ,a multimedia presentation algebra,"s adali, m sapino, v subrahmanian",1999,a multimedia presentation algebra,"s adah, ml sapino, vs subrahmanian",1999,1
7,161,161,conf/sigmod/Adelberg98,JrU1gnfH1sYJ,nodose - a tool for semi-automatically extracting semi-structured data from text documents,b adelberg,1998,nodose {a tool for semi-automatically extracting semi-structured data from text,b adelberg,0,1
8,167,167,conf/sigmod/AdelbergGK95,b87rz7o7R94J,applying update streams in a soft real-time database system,"b adelberg, h garcia-molina, b kao",1995,rules in strip,"b adelberg, h garcia-molina, b kao",0,0
9,208,208,conf/sigmod/AgichteinG03,1HwyWUB6M50J,qxtract: a building block for efficient information extraction from plain-text databases,"e agichtein, l gravano",2003,querying text databases for efficient information extraction,"e agichtein, l gravano",0,0


In [5]:
i_file = "datasets/DBLP-Scholar/I.csv"
j_file = "datasets/DBLP-Scholar/J.csv"

if not os.path.isfile(i_file):
    IJ = em.split_train_test(S, train_proportion=0.5, random_state=0)
    I = IJ['train']
    J = IJ['test']
    I.to_csv(i_file, sep=",")
    J.to_csv(j_file, sep=",")
    print("Split samples into I and J")
else:
    I = em.read_csv_metadata(i_file, key="_id",
                             ltable=dblp, rtable=scholar,
                             fk_ltable="ltable_id", fk_rtable='rtable_id')
    J = em.read_csv_metadata(j_file, key="_id",
                             ltable=dblp, rtable=scholar,
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
    print("Reading I and J from files")                         
print(len(I))
print(len(J))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Reading I and J from files
250
250


## Matching(匹配)

In [6]:
em.set_key(dblp, 'id')
em.set_key(scholar, 'id')

True

In [7]:
F = em.get_features_for_matching(dblp, scholar, validate_inferred_attr_types=False)

In [8]:
H = em.extract_feature_vecs(I, feature_table=F, attrs_after='label', show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


## 创建学习器

In [9]:
import random
random_state = 0 

dt = em.DTMatcher(name='DecisionTree', random_state=random_state)
rf = em.RFMatcher(name='RF', random_state=random_state)
svm = em.SVMMatcher(name='SVM', random_state=random_state)
ln = em.LinRegMatcher(name='LinReg')
lg = em.LogRegMatcher(name='LogReg', random_state=random_state)
nb = em.NBMatcher(name = 'NaiveBayes')

In [10]:
# Impute feature vectors with the mean of the column values.
# 用列的平均值填充特征向量
H = em.impute_table(H, exclude_attrs=['_id', 'ltable_id','rtable_id','label'], strategy='mean')

/home/madao/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
H.head()

,_id,ltable_id,rtable_id,id_id_lev_dist,id_id_lev_sim,id_id_jar,id_id_jwn,id_id_exm,id_id_jac_qgm_3_qgm_3,title_title_jac_qgm_3_qgm_3,...,venue_venue_mel,venue_venue_lev_dist,venue_venue_lev_sim,venue_venue_nmw,venue_venue_sw,year_year_exm,year_year_anm,year_year_lev_dist,year_year_lev_sim,label
0,9171,journals/vldb/GutingS95,mGDi-Lnj27QJ,23.0,0.000000,0.251208,0.251208,0.0,0.0,0.405941,...,0.379572,66.0,0.043478,-59.0,1.0,0.632653,0.998868,2.870748,0.521542,0
1,2725,conf/sigmod/ZhangRL96,46aixXBMD6sJ,20.0,0.047619,0.253968,0.253968,0.0,0.0,0.681319,...,0.593867,33.0,0.297872,-16.0,7.0,0.632653,0.998868,2.870748,0.521542,1
2,634,conf/sigmod/ChakrabartiDI98,AyDb4G2EiGwJ,26.0,0.037037,0.373457,0.373457,0.0,0.0,0.071429,...,0.479163,59.0,0.119403,-42.0,3.0,1.000000,1.000000,2.000000,0.666667,0
3,799,conf/sigmod/ChristophidesCS00,DqHBAYRAtyIJ,28.0,0.034483,0.372605,0.372605,0.0,0.0,0.594595,...,0.590756,54.0,0.228571,-37.0,10.0,1.000000,1.000000,2.000000,0.666667,1
4,1601,conf/sigmod/Kim94,JHuJHooGoNUJ,16.0,0.058824,0.428105,0.428105,0.0,0.0,0.024390,...,0.480838,40.0,0.090909,-23.0,2.0,0.000000,0.999499,3.000000,0.500000,0


In [12]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb],       table=H, exclude_attrs=['_id', 'ltable_id',             'rtable_id', 'label'], k=5,target_attr='label',
    metric_to_select_matcher='f1',
    random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.863046,0.882635,0.867502
1,RF,0.920724,0.878447,0.895470
2,SVM,0.891378,0.672591,0.761804
3,LinReg,0.847413,0.895039,0.865603
4,LogReg,0.885790,0.875326,0.875706
5,NaiveBayes,0.896039,0.866901,0.878164


In [13]:
result['drill_down_cv_stats']['precision']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f55fb9a6a58>,5,0.947368,0.806452,0.791667,0.838710,0.931034,0.863046
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f55fb9a6400>,5,1.000000,0.888889,0.818182,0.896552,1.000000,0.920724
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f55fb9a6e80>,5,0.857143,0.863636,0.777778,0.958333,1.000000,0.891378
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f55fb9a6630>,5,0.894737,0.833333,0.730769,0.875000,0.903226,0.847413
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f55fb9a6550>,5,0.947368,0.827586,0.760000,0.931034,0.962963,0.885790
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f55fb9a60b8>,5,0.950000,0.862069,0.782609,0.920000,0.965517,0.896039


In [14]:
result['drill_down_cv_stats']['recall']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f55fb9a6a58>,5,0.750000,1.00,0.863636,0.928571,0.870968,0.882635
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f55fb9a6400>,5,0.750000,0.96,0.818182,0.928571,0.935484,0.878447
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f55fb9a6e80>,5,0.500000,0.76,0.636364,0.821429,0.645161,0.672591
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f55fb9a6630>,5,0.708333,1.00,0.863636,1.000000,0.903226,0.895039
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f55fb9a6550>,5,0.750000,0.96,0.863636,0.964286,0.838710,0.875326
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f55fb9a60b8>,5,0.791667,1.00,0.818182,0.821429,0.903226,0.866901


In [15]:
result['drill_down_cv_stats']['f1']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0x7f55fb9a6a58>,5,0.837209,0.892857,0.826087,0.881356,0.900000,0.867502
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0x7f55fb9a6400>,5,0.857143,0.923077,0.818182,0.912281,0.966667,0.895470
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0x7f55fb9a6e80>,5,0.631579,0.808511,0.700000,0.884615,0.784314,0.761804
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0x7f55fb9a6630>,5,0.790698,0.909091,0.791667,0.933333,0.903226,0.865603
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0x7f55fb9a6550>,5,0.837209,0.888889,0.808511,0.947368,0.896552,0.875706
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0x7f55fb9a60b8>,5,0.863636,0.925926,0.800000,0.867925,0.933333,0.878164


### Train classfiers on I and test on J(利用I样本训练和利用J样本训练)

In [16]:
classifiers = np.array([dt, rf, svm, ln, lg, nb])

In [17]:
# Convert J into a set of feature vectors using F
# 利用F将J转换为特征向量
L = em.extract_feature_vecs(J, feature_table=F, attrs_after='label', show_progress=True)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [18]:
# Impute feature vectors with the mean of the column values
# 利用每一栏的平均值填充特征向量
L = em.impute_table(L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], strategy='mean')

/home/madao/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
for c in classifiers:
    # Train using feature vectors from I
    # 利用来自I的特征向量训练
    c.fit(table=H, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], target_attr='label')

    # Predict on L(在L上预测)
    predictions = c.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'label'], append=True, target_attr='predicted', inplace=False)

    predictions[['_id', 'ltable_id', 'rtable_id', 'predicted', 'label']].head()

    print(c.name)
    eval_result = em.eval_matches(predictions, 'label', 'predicted')
    em.print_eval_summary(eval_result)
    print()

DecisionTree
Precision : 86.36% (114/132)
Recall : 84.44% (114/135)
F1 : 85.39%
False positives : 18 (out of 132 positive predictions)
False negatives : 21 (out of 118 negative predictions)

RF
Precision : 93.7% (119/127)
Recall : 88.15% (119/135)
F1 : 90.84%
False positives : 8 (out of 127 positive predictions)
False negatives : 16 (out of 123 negative predictions)

SVM
Precision : 94.0% (94/100)
Recall : 69.63% (94/135)
F1 : 80.0%
False positives : 6 (out of 100 positive predictions)
False negatives : 41 (out of 150 negative predictions)

LinReg
Precision : 86.21% (125/145)
Recall : 92.59% (125/135)
F1 : 89.29%
False positives : 20 (out of 145 positive predictions)
False negatives : 10 (out of 105 negative predictions)

LogReg
Precision : 87.86% (123/140)
Recall : 91.11% (123/135)
F1 : 89.45%
False positives : 17 (out of 140 positive predictions)
False negatives : 12 (out of 110 negative predictions)

NaiveBayes
Precision : 87.86% (123/140)
Recall : 91.11% (123/135)
F1 : 89.45%
False